This is an abridged version of the full notebook. Since the full notebook has gotten unbearably long and searching for errors is infeasible I have created this shortened one which should help.

# [0] Imports

In [343]:
import Pkg;

using LinearAlgebra, Random, Gurobi, JuMP, Distributions, Plots, LazySets

# [1] Setup

## [1.1] Parameters

In [344]:
n_jobs = 25
n_vehicles_jobs = 25
n_vehicles_coverage = 10
T = 50
min_duration = 2
max_duration = 6
speed = 1000/6
coverage_distance = 50
size = 500
step = 50;

## [1.2] Locations, Windows, Loads

In [345]:
Random.seed!(1234)

function create_cluster_sizes(jobs_total)
    Random.seed!(1234)
    jobs_created = 0
    cluster = []
    while jobs_created != jobs_total
        num_to_add = rand(min(3, jobs_total - jobs_created) : 
            min(Int((n_jobs/5)÷1), jobs_total - jobs_created))
        jobs_created += num_to_add
        push!(cluster, num_to_add)
    end
    
    return cluster
end;

In [346]:
time_windows = []
locations = rand(Uniform(0,size), 1, 2)
work_load = []
cluster = create_cluster_sizes(n_jobs);

In [347]:
function create_time_windows_and_work_load(cluster_sizes, locations)
    Random.seed!(1234)
    locations = rand(Uniform(0,size), 1, 2)
    for size_c in cluster
        first = rand(Uniform(0,size), 1, 2)
        locations = vcat(locations, first)

        job_begins = rand(2:10)
        job_finish = rand((job_begins+min_duration):(job_begins+max_duration))
        push!(time_windows, [job_begins, job_finish])

        time_work = rand(min_duration:max(min_duration, job_finish - job_begins))
        push!(work_load, time_work)

        for neighbour in 1:(size_c-1)
            new_x = rand(Uniform(max(0,first[1]-20), min(first[1]+20, size)), 1, 1)
            new_y = rand(Uniform(max(0,first[2]-20), min(first[2]+20, size)), 1, 1)
            new = hcat(new_x, new_y)
            locations = vcat(locations, new)

            job_begins = rand(job_finish:min(T-min_duration-2, job_finish + 6))
            job_finish = rand((job_begins+min_duration):(min(job_begins+max_duration, T-2)))
            push!(time_windows, [job_begins, job_finish])

            time_work = rand(min_duration:min(max_duration, job_finish-job_begins))
            push!(work_load, time_work)
        end
    end
    
    return [time_windows, work_load, locations]
end;

In [348]:
time_windows, work_load, locations = create_time_windows_and_work_load(cluster, locations)
distances = [LinearAlgebra.norm(locations[i, :] .- locations[j, :]) for i=1:n_jobs+1, j = 1:n_jobs+1];

# [2] Helper Methods

## 2.1 Get initial routes

In [349]:
routes = []

for n in 1:n_jobs
    job_route = []
    dist = distances[1, n+1]

    min_t = Int(floor(time_windows[n][1] - dist/speed))
    max_t = Int(ceil(time_windows[n][2] + dist/speed))

    push!(job_route, [[0, min_t], [n, time_windows[n][1]], dist])
    
    for t in time_windows[n][1]:(time_windows[n][2]-1)
        push!(job_route, [[n, t], [n, t+1], 0])
    end
    
    push!(job_route, [[n, time_windows[n][2]], [n_jobs+1, max_t],  dist])
    push!(routes, job_route)
    
end

## 2.2 Compute Cost

In [350]:
function compute_cost(route)
    cost = 0
    for entity in route
        cost += entity[3]
    end
    return cost
end;

In [351]:
C = []
for i in 1:length(routes)
    push!(C, compute_cost(routes[i]))
end

## 2.3 Compute Delta

In [352]:
function compute_delta(routes)
    Q = length(routes)
    delta = [[[0.0 for q in 1:Q] for t in 1:T] for i in 1:n_jobs]
    for rindex in 1:Q
        route = routes[rindex]
        for arc in route
            loc1, time1 = arc[1]
            loc2, time2 = arc[2]
            if (loc1 != 0) & (loc1 != n_jobs + 1)
                delta[loc1][time1][rindex] = 1
            end
            
            if (loc2 != 0) & (loc2 != n_jobs + 1)
                delta[loc2][time2][rindex] = 1
            end
        end
    end
    return delta
end;

In [353]:
delta = compute_delta(routes);

## 2.4 Compute U

In [354]:
function compute_u(routes)
    Q = length(routes)
    u = [[0 for q in 1:Q] for i in 1:n_jobs]
    
    for rindex in 1:Q
        route = routes[rindex]
        for arc in route
            loc1 = arc[1][1]
            loc2 = arc[2][1]
            if (1 <= loc1 <= n_jobs)
                u[loc1][rindex] = 1
            end
            if (1 <= loc2 <= n_jobs)
                u[loc2][rindex] = 1
            end
        end
    end
    return u
end;

In [355]:
u = compute_u(routes);

## 2.5 Label-Setting Algorithm

In [356]:
function sp_lsa(n, travel_distance, travel_time, windows, load, rho_v, pi_v, mu_v, cool_stuff=false)
    N = [[1]] 
    T = [ [0] ] 
    R = [rho_v]  
    L = [1]  
    A = [true] 
    
    #= Two parameters which are good if you're visualizing
    how quickly the number if iterations catches up
    to the total number of routes generated
    =#
    current_states = []
    total_states = []
    
    current_state = 1
    total_state = 1
    
    #=
    NOTE: For pi_v, u_v, mu_v, and delta_v, there is no defined value
    for the actual depot. We will be using -1's for indexing quite liberally,
    as in our loop, 1 = n+2 = depot, when in 'reality', this should be 0 = n+!.
    
    This is also the case for variables windows and load. This happened last LSA too.
    =#
    
    while 1==1
        
        # STEP 1: Check whether we want to move on to checking the next path
        if (L[current_state] == n+2) | (~A[current_state])
            current_state += 1
            if current_state > total_state
                print(current_state, " ", total_state, " nothing more to check")
                break
            else
                continue
            end
        end
        
        # STEP 2: Enter the for loop.
        for i in 2:(n+2)
            
            if cool_stuff
                push!(current_states, current_state)
                push!(total_states, total_state)
            end
            
            # STEP 2-1: Check that the current node is not already inside our path.
            if ~(i in N[current_state])
                
                #= STEP 2-2: Feasibility check. We must have enough time
                to go from our current location (end time T[current_state]), plus
                the travel time from that end node to the new node (travel_time[L[current_state], i]),
                plus the amount of time it takes to work the OLD job (load[L[current_state]-1]),
                plus the amount of time it takes to work the NEW job (load[i-1])
                
                this must be smaller than or equal to the end time of the window for the NEW job (windows[i-1][2])
                =#
                if L[current_state] != 1
                    cur_time = last(T[current_state])
                    dist_nec = travel_time[L[current_state], i]
                    old_job_time_nec = load[L[current_state]-1] #out of our parameters this one has no depot pad
                    new_job_time_nec = load[i-1]
                    new_window_close = windows[i-1][2]
                    
                    #if invalid, don't bother
                    if cur_time + dist_nec + old_job_time_nec + new_job_time_nec > new_window_close
                        continue
                    end
                end
                
                #= STEP 2-3: By the design of our algorithm, sometimes we create a new and
                invalid path on index total_state + 1. Either we did create that
                invalid path, or we did not. But the goal of entering this inner for loop
                remains the same: we want to add something to our path.
                =#
                
                # STEP 2-3-1: We did not create an invalid path earlier
                if length(N) < total_state + 1
                    
                    # First, copy the original state so we can modify it
                    push!(N, copy(N[current_state]))
                    # Add this new, guaranteed-to-be-valid, node.
                    push!(N[total_state+1], i)
                    
                    # Time is going to be when we GET to the final node.
                    
                    #= I argue that we also need to include a work load term into the 
                    term bounded by current_time + necessary_travel_distance.
                    
                    This is because there's no point in going to an old place, not 
                    doing any work, and then going away to some new place.
                    
                    So in reality the check becomes tripartite.
                    =#
                    cur_time = last(T[current_state])
                    dist_nec = travel_time[L[current_state], i]
                    old_job_time = L[current_state] > 1 ? load[L[current_state]-1] : 0
                    new_window_start = windows[i-1][1]
                    new_times_array = copy(T[current_state])
                    push!(new_times_array, max(cur_time + dist_nec + old_job_time, new_window_start))
                    push!(T, new_times_array)
                    
                    # Cost.
                    # ADD the segment distance.
                    add_segdist = travel_distance[L[current_state], i]
                    # SUBTRACT pi_i.
                    subtract_pi = i < n+2 ? -pi_v[i-1] : 0
                    # SUBTRACT mu_it.
                    subtract_mu = i < n+2 ? -mu_v[i-1, last(last(T))] : 0
                    
                    push!(R, R[current_state] + add_segdist + subtract_pi + subtract_mu)
                    
                    # Last node is updated.
                    push!(L, i)
                    
                    # Update feasibility.
                    push!(A, true)
                    
                # STEP 2-3-2: We created an invalid path earlier
                else
                    # Change the path
                    N[total_state + 1] = N[current_state]
                    push!(N[total_state+1], i)
                    
                    # Change the time
                    cur_time = last(T[current_state])
                    dist_nec = travel_time[L[current_state], i]
                    old_job_time = L[current_state] > 1 ? load[L[current_state]-1] : 0
                    new_window_start = windows[i-1][1]
                    new_times_array = copy(T[current_state])
                    push!(new_times_array, max(cur_time + dist_nec + old_job_time, new_window_start))
                    T[total_state+1] = copy(new_times_array)
                    
                    # Change the cost - watch out for if we're going back to depot on pi and mu!
                    add_segdist = travel_distance[L[current_state], i]
                    subtract_pi = i < n+2 ? -pi_v[i-1] : 0
                    subtract_mu = i < n+2 ? -mu_v[i-1, last(last(T))] : 0
                    R[total_state + 1] = R[current_state] + add_segdist + subtract_pi + subtract_mu
                    
                    # Change the last node
                    L[total_state+1] = i
                    
                    # Change the feasibility
                    A[total_state+1] = true
                end 
                
                # STEP 2-4: Reviewing true Bellman-Ford step: if a path is totally dominated by another.
                NDom = true
                
                #= Iterate over all old paths and check if they have the same start (guaranteed)
                and end point. If they do, check which cost is lower. =#
                for s in 1:total_state
                    # Is the route feasible?
                    if A[s]
                        # if so, do they share the same start and end?
                        if (issetequal(N[total_state+1], N[s])) & (L[total_state+1] == L[s])
                            # New path is too bad, overwrite it later
                            if R[total_state+1] > R[s]
                                NDom = false
                                break
                            # Old path is too bad, declare it not valid and proceed to NDom add total state
                            else
                                A[s] = false
                            end
                        end
                    end
                end
                
                # New state good enough, it's set in stone and we add
                if NDom 
                    total_state += 1
                end
        
            end
        end
        
        current_state += 1 # concluded iteration for that state in mind. Now next one
        
        #= It's possible that we've just caught up to the final total state, meaning we didn't generate
        anything and can stop here. =#
        if current_state == total_state
            print(current_state, " ", total_state, " came from bottom")
            break
        end     
    end
    
    #= It is not actually necessary to return L. This was a helpful parameter which we
    embedded in order to aid computation. But if we needed to get L, we would just go to 
    the path of concern and extract last([path_of_concern]). =#
    
    if ~cool_stuff
        return N, A, R, T
    else
        return N, A, R, T, current_states, total_states
    end
end

sp_lsa (generic function with 2 methods)

### 2.5.1 Label-Setting Algorithm Constant Variables

In [357]:
distances_label = deepcopy(distances)
distances_label = hcat(distances_label, distances_label[:, 1])
distances_label = vcat(distances_label, collect(push!(distances_label[:, 1], 0)'));

In [358]:
travel_times = ceil.(distances_label / speed);

In [359]:
windows_label = deepcopy(time_windows)
push!(windows_label, [0, 100]);

In [360]:
load_label = deepcopy(work_load)
push!(load_label, 0);

## 2.6 Extract Best Route in SP

In [361]:
function extract_best_route(sub_paths, sub_bool, sub_rc, sub_times)
    best_index = 1
    best_cost = 1

    for i in 1:length(sub_paths)

        ends_at_depot = (last(sub_paths[i]) == n_jobs + 2)
        negative_reduced_cost = (sub_rc[i] < -1e-5)
        time_fine = (last(sub_times[i]) <= T)

        if (ends_at_depot && negative_reduced_cost && time_fine && sub_bool[i])
            if sub_rc[i] < best_cost
                best_index = i
                best_cost = sub_rc[i]
            end
        end
    end

    if best_cost >= -1e-5
        println("We have concluded our column generation because nothing better is to be found!")
        println("The solution you found above is optimal.")
        return
    end
    
    return sub_paths[best_index], sub_rc[best_index], sub_times[best_index]
end;

## 2.7 Generate full time info on route

In [362]:
function generate_full_time_info(best_route, best_times)
    best_route_info = []
    for locindex in 1:length(best_route)-1
        #=
        Strategy:
        If you are not at a job location, you don't need to capture
        stationary information. Otherwise you do.

        Anyways we want data moving from one to the next.
        =#
        cur_loc = best_route[locindex]
        cur_time = best_times[locindex]
        time_to_start_moving = best_times[locindex]

        #stationary data comes first.
        if 1 < locindex < n_jobs + 2
            cur_job_num = cur_loc - 1 # delete padding!

            #stay stationary for however long it takes for you to do the job

            # Note: eventually this will have to be modified to include coverage.
            # Eventually we will write a subproblem-tells-us-when-work-is-done component.

            for time in cur_time:cur_time+work_load[cur_job_num]-1
                route_detail = [ [cur_loc-1, time], [cur_loc-1, time+1], 0]
                push!(best_route_info, route_detail)
            end

            time_to_start_moving += work_load[cur_job_num]
        end

        new_loc = best_route[locindex+1]
        #then if we move, we move.
        route_detail = [ [cur_loc-1, time_to_start_moving], [new_loc-1, best_times[locindex+1]], distances_label[cur_loc, new_loc]]
        push!(best_route_info, route_detail)
    end

    return best_route_info
end;

# [3] Column Generation Loop

## [3.1] Initial Run

### [3.1.1] Run RMP

In [363]:
modelcg = Model(Gurobi.Optimizer);

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-04


In [473]:
Q = length(routes)

30

In [474]:
unregister(modelcg, :z)
unregister(modelcg, :y_s)
unregister(modelcg, :y_e)
@variable(modelcg, 0 <= z[1:Q] <= 1) # should be Bin
@variable(modelcg, 0 <= y_s[1:n_jobs, 1:T] <= 1) # should be Bin
@variable(modelcg, 0 <= y_e[1:n_jobs, 1:T] <= 1); # should be Bin

In [475]:
unregister(modelcg, :unique)
@constraint(modelcg, unique[i in 1:n_jobs], sum(u[i][q] * z[q] for q in 1:Q) >= 1);

In [476]:
unregister(modelcg, :driver)
@constraint(modelcg, driver, sum(z[q] for q in 1:Q) <= n_vehicles_jobs)

unregister(modelcg, :start)
unregister(modelcg, :ends)
@constraint(modelcg, start[i in 1:n_jobs, t in 1:(T-1)], y_s[i,t] <= y_s[i,t+1])
@constraint(modelcg, ends[i in 1:n_jobs, t in 1:(T-1)], y_e[i,t] <= y_e[i,t+1])
unregister(modelcg, :window_start)
unregister(modelcg, :window_ends)
@constraint(modelcg, window_start[i in 1:n_jobs, t in 1:(time_windows[i][1]-1)], y_s[i,t] == 0)
@constraint(modelcg, window_ends[i in 1:n_jobs, t in time_windows[i][2]:T], y_e[i,t] == 1)
unregister(modelcg, :duration)
@constraint(modelcg, duration[i in 1:n_jobs], sum(y_s[i,t] - y_e[i,t] for t in 1:T) >= work_load[i])
unregister(modelcg, :work)
@constraint(modelcg, work[i in 1:n_jobs, t in 1:(T-1)], 
    y_s[i, t] - y_e[i, t] <= sum(z[q] * delta[i][t][q] for q in 1:Q));

In [477]:
unregister(modelcg, :coverage)
@constraint(modelcg, coverage[i in 1:n_jobs, t in 1:T], y_s[i, t] - y_e[i, t] <= 1);

In [478]:
@objective(modelcg, Min, sum(compute_cost(routes[q]) * z[q] for q in 1:Q));

In [482]:
optimize!(modelcg)

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 36756 rows, 15165 columns and 82510 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  4.670168240e+03

User-callback calls 12, time in user-callback 0.00 sec


### [3.1.2] Get Dual Variables

In [483]:
pi_values = dual.(unique)
rho_value = dual.(driver)
mu_values = dual.(work);

### [3.1.3] Use Subproblem LSA

In [485]:
sub_paths, sub_bool, sub_rc, sub_times = sp_lsa(n_jobs, distances_label, travel_times, windows_label, load_label, rho_value, pi_values, mu_values, true);

LoadError: InterruptException:

### [3.1.4] Extract the best route with most negative reduced cost

In [468]:
best_route, best_rc, best_times = extract_best_route(sub_paths, sub_bool, sub_rc, sub_times);

### [3.1.5] Generate full time info on best route

In [469]:
best_route_info = generate_full_time_info(best_route, best_times);

### [3.1.6] Add best route info to Q

In [470]:
push!(routes, best_route_info);

### [3.1.7] Update Parameters

In [471]:
u = compute_u(routes)
delta = compute_delta(routes);

In [472]:
C = []
for i in 1:length(routes)
    push!(C, compute_cost(routes[i]))
end;